In [1]:
import os, time, requests
URL = "http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.txt"
##URL = "https://s3.eu-west-1.amazonaws.com/prod1.publicdata.landregistry.gov.uk/pp-2025.txt"
OUT = "pp-2025.txt"

def download_resume(url, out_path, retries=20, chunk_size=1024*1024):
    s = requests.Session()
    headers = {"User-Agent": "Mozilla/5.0", "Accept-Encoding": "identity"}
    downloaded = os.path.getsize(out_path) if os.path.exists(out_path) else 0

    for attempt in range(1, retries + 1):
        try:
            h = headers.copy()
            if downloaded:
                h["Range"] = f"bytes={downloaded}-"

            with s.get(url, stream=True, headers=h, timeout=(30, 300)) as r:
                if r.status_code == 416:
                    print("Already complete.")
                    return
                r.raise_for_status()

                mode = "ab" if downloaded else "wb"
                with open(out_path, mode) as f:
                    last = time.time()
                    for chunk in r.iter_content(chunk_size=chunk_size):
                        if not chunk:
                            continue
                        f.write(chunk)
                        downloaded += len(chunk)
                        if time.time() - last > 2:
                            print(f"\rDownloaded {downloaded/1e6:,.1f} MB", end="")
                            last = time.time()

            print(f"\nDone: {out_path}")
            return

        except (requests.Timeout, requests.ConnectionError) as e:
            wait = min(2 ** attempt, 60)
            print(f"\nAttempt {attempt} failed: {e} | retrying in {wait}s (resume at {downloaded/1e6:,.1f} MB)")
            time.sleep(wait)

    raise RuntimeError("Failed after retries")

download_resume(URL, OUT)

Downloaded 5,100.3 MB
Done: pp-2025.txt


In [2]:
import os
print(os.getcwd())
print(os.listdir(".")[:50])

/workspaces/valuemap-uk/pipeline
['pp-2025.txt', 'build_grids.ipynb', 'Untitled.ipynb', 'build_grids.py', '.ipynb_checkpoints']


In [7]:
from pathlib import Path

ROOT = Path.cwd()   # start here
# if you're inside notebooks/, go up one:
if (ROOT / "package.json").exists() is False and (ROOT.parent / "package.json").exists():
    ROOT = ROOT.parent

DATA = ROOT / "data"
DATA.mkdir(exist_ok=True)

In [9]:
import os
print("CWD:", os.getcwd())
print("Here:", os.listdir(".")[:50])

CWD: /workspaces/valuemap-uk/pipeline
Here: ['pp-2025.txt', 'build_grids.ipynb', 'Untitled.ipynb', 'build_grids.py', '.ipynb_checkpoints']


In [11]:
import os

for d in os.listdir("data"):
    p = f"data/{d}"
    print("\n==", p)
    print(os.listdir(p)[:50])

In [13]:
import pandas as pd

path = "pp-2025.txt"

# Full PPD schema (16 columns)
all_cols = [
    "transaction_id", "price", "date", "postcode",
    "property_type", "new_build", "tenure",
    "paon", "saon", "street", "locality", "town_city",
    "district", "county", "ppd_category", "record_status"
]

usecols = ["transaction_id", "price", "date", "postcode", "property_type", "new_build", "record_status"]

cutoff = (pd.Timestamp.today().normalize() - pd.DateOffset(years=5)).date()

chunks = []
for chunk in pd.read_csv(
    path,
    header=None,
    names=all_cols,
    usecols=usecols,
    dtype={
        "transaction_id": "string",
        "postcode": "string",
        "property_type": "string",
        "new_build": "string",
        "record_status": "string",
    },
    chunksize=500_000
):
    # Parse dates for this chunk
    chunk["date"] = pd.to_datetime(chunk["date"], errors="coerce")
    chunk = chunk[chunk["date"].notna()]

    # Keep only "A" records (usual rule)
    chunk = chunk[chunk["record_status"] == "A"]

    # Last 5 years
    chunk = chunk[chunk["date"].dt.date >= cutoff]

    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

# Month-start timestamp (for your downstream logic)
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

print("Rows kept:", len(df))
print("Date range:", df["date"].min().date(), "->", df["date"].max().date())

Rows kept: 4705970
Date range: 2021-02-06 -> 2025-12-24


In [14]:
df.head()

,transaction_id,price,date,postcode,property_type,new_build,record_status,month
0,{D707E535-5720-0AD9-E053-6B04A8C067CC},260000,2021-08-06,SO45 2HT,T,N,A,2021-08-01
1,{D707E535-5721-0AD9-E053-6B04A8C067CC},375000,2021-09-01,SO23 7FR,S,N,A,2021-09-01
2,{D707E535-5723-0AD9-E053-6B04A8C067CC},132000,2021-06-28,SP11 6RL,F,N,A,2021-06-01
3,{D707E535-5724-0AD9-E053-6B04A8C067CC},295000,2021-09-10,SP11 6TU,T,N,A,2021-09-01
4,{D707E535-5725-0AD9-E053-6B04A8C067CC},360000,2021-08-27,SO51 0AX,T,N,A,2021-08-01


In [ ]:
import pandas as pd
path_east_north = "/kaggle/input/postcode-eastnorth/ONSPD_Online_latest_Postcode_Centroids_.csv"

cols = [
    "x","y","PCD7","PCD8","PCDS","DOINTR","DOTERM",
    "EAST1M","NORTH1M"
]

df_en = pd.read_csv(
    path_east_north,    
    names=cols,
    header=None,
    usecols=cols,
    skiprows=1,
    dtype={
        "x": "string",
        "y": "string",
        "PCD7": "string",
        "PCD8": "string",
        "PCDS": "string",
        "DOINTR": "string",
        "DOTERM": "string",
        "EAST1M": "string",
        "NORTH1M": "string",        
    }
)

df_en.head()

In [ ]:
df_en["EAST1M"]  = pd.to_numeric(df_en["x"], errors="coerce")
df_en["NORTH1M"] = pd.to_numeric(df_en["y"], errors="coerce")

df_en = df_en.dropna(subset=["EAST1M","NORTH1M"]).copy()

In [ ]:
GRID_SIZES = [1000, 5000, 10000, 25000]  # metres

for g in GRID_SIZES:
    df_en[f"gx_{g}"] = (df_en["EAST1M"] // g) * g
    df_en[f"gy_{g}"] = (df_en["NORTH1M"] // g) * g
    df_en[f"cell_{g}"] = (
        df_en[f"gx_{g}"].astype("Int64").astype(str) + "_" +
        df_en[f"gy_{g}"].astype("Int64").astype(str)
    )

In [ ]:
def make_cells(df, g):
    cells = (df[[f"gx_{g}", f"gy_{g}"]]
             .drop_duplicates()
             .rename(columns={f"gx_{g}":"gx", f"gy_{g}":"gy"}))
    cells["grid_m"] = g
    cells["x0"] = cells["gx"]
    cells["y0"] = cells["gy"]
    cells["x1"] = cells["gx"] + g
    cells["y1"] = cells["gy"] + g
    return cells

cells_1km  = make_cells(df_en, 1000)
cells_5km  = make_cells(df_en, 5000)
cells_10km = make_cells(df_en, 10000)
cells_25km = make_cells(df_en, 25000)

In [ ]:
print("1km:",  len(cells_1km))
print("5km:",  len(cells_5km))
print("10km:", len(cells_10km))
print("25km:", len(cells_25km))

In [ ]:
df["pc_key"] = df["postcode"].astype("string").str.replace(" ", "", regex=False).str.upper()
df_en["pc_key"] = df_en["PCDS"].astype("string").str.replace(" ", "", regex=False).str.upper()

In [ ]:
df_en.head()

In [ ]:
df_en["EAST1M"]  = pd.to_numeric(df_en["EAST1M"], errors="coerce")
df_en["NORTH1M"] = pd.to_numeric(df_en["NORTH1M"], errors="coerce")
df_en = df_en.dropna(subset=["EAST1M","NORTH1M"]).copy()

for g in [1000, 5000, 10000, 25000]:
    df_en[f"gx_{g}"] = ((df_en["EAST1M"] // g) * g).astype("int64")
    df_en[f"gy_{g}"] = ((df_en["NORTH1M"] // g) * g).astype("int64")
    df_en[f"cell_{g}"] = df_en[f"gx_{g}"].astype(str) + "_" + df_en[f"gy_{g}"].astype(str)

In [ ]:
lookup_cols = [
    "pc_key",
    "EAST1M", "NORTH1M",
    "gx_1000", "gy_1000", "cell_1000",
    "gx_5000", "gy_5000", "cell_5000",
    "gx_10000", "gy_10000", "cell_10000",
    "gx_25000", "gy_25000", "cell_25000",
]

lookup = df_en[lookup_cols].drop_duplicates("pc_key")

to_drop = [c for c in lookup_cols if c in df.columns and c != "pc_key"]
df = df.drop(columns=to_drop)

df = df.merge(lookup, on="pc_key", how="left")

# --- Postcode area (outcode) -> grid cell lookup tables (for UI drilldown) ---
# Keep only the outcode (part before the space) to reduce size.
postcode_lookup = df_en[[
    "PCDS",
    "cell_1000", "cell_5000", "cell_10000", "cell_25000",
]].copy()
postcode_lookup["outcode"] = (
    postcode_lookup["PCDS"].astype("string").str.strip().str.split(" ", n=1).str[0].str.upper()
)
postcode_lookup = postcode_lookup.dropna(subset=["outcode"]).drop_duplicates(
    ["outcode", "cell_1000", "cell_5000", "cell_10000", "cell_25000"]
)

# Save as parquet (compact, fast) and JSON (portable)
postcode_lookup_out_parquet = "/kaggle/working/postcode_grid_outcode_lookup.parquet"
postcode_lookup_out_json = "/kaggle/working/postcode_grid_outcode_lookup.json.gz"

postcode_lookup.to_parquet(postcode_lookup_out_parquet, index=False)

import json, gzip
with gzip.open(postcode_lookup_out_json, "wt", encoding="utf-8") as f:
    json.dump(postcode_lookup.where(pd.notnull(postcode_lookup), None).to_dict(orient="records"), f)

print("Wrote postcode lookup:", postcode_lookup_out_parquet, "rows:", len(postcode_lookup))

In [ ]:
df[["postcode", "EAST1M", "NORTH1M", "gx_25000", "gy_25000", "cell_25000"]].head()

In [ ]:
df["month"] = df["date"].dt.to_period("M").dt.to_timestamp()

# optional but recommended for later
df["property_type"] = df["property_type"].astype("string")
df["new_build"] = df["new_build"].astype("string")

In [ ]:
df["month"] = pd.to_datetime(df["month"]).dt.to_period("M").dt.to_timestamp()
latest_month = df["month"].max()
# Keep only last 10 years (inclusive, aligned to month)
cutoff_month = (latest_month - pd.DateOffset(years=10)).to_period("M").to_timestamp()
df = df[df["month"] >= cutoff_month].copy()

In [ ]:
import pandas as pd

def _yearly_end_months(d_month_col: pd.Series, years_back: int):
    latest = d_month_col.max()
    end_months = [latest]
    cursor = latest - pd.DateOffset(years=1)
    min_month = d_month_col.min()
    while cursor >= min_month and len(end_months) < (years_back + 1):
        end_months.append(cursor)
        cursor -= pd.DateOffset(years=1)
    return end_months

def make_grid_annual_stack_levels(df, g, min_sales=3, years_back=10):
    """
    Returns ONE dataframe containing 4 levels:
      - TYPE+BUILD
      - TYPE+ALL
      - ALL+BUILD
      - ALL+ALL
    All use true medians (transaction-level) over trailing 12 months.
    """
    gx, gy = f"gx_{g}", f"gy_{g}"

    d = df.dropna(subset=[gx, gy]).copy()
    d["month"] = pd.to_datetime(d["month"]).dt.to_period("M").dt.to_timestamp()

    end_months = _yearly_end_months(d["month"], years_back=years_back)

    parts = []
    for end_month in end_months:
        start_month = (end_month - pd.DateOffset(months=11)).to_period("M").to_timestamp()
        w = d[(d["month"] >= start_month) & (d["month"] <= end_month)]

        # 1) TYPE + BUILD
        a = (w.groupby([gx, gy, "property_type", "new_build"], as_index=False)
               .agg(median_price_12m=("price", "median"),
                    sales_12m=("price", "size")))
        a["end_month"] = end_month
        parts.append(a)

        # 2) TYPE + ALL_BUILD
        b = (w.groupby([gx, gy, "property_type"], as_index=False)
               .agg(median_price_12m=("price", "median"),
                    sales_12m=("price", "size")))
        b["end_month"] = end_month
        b["new_build"] = "ALL"
        parts.append(b)

        # 3) ALL_TYPE + BUILD
        c = (w.groupby([gx, gy, "new_build"], as_index=False)
               .agg(median_price_12m=("price", "median"),
                    sales_12m=("price", "size")))
        c["end_month"] = end_month
        c["property_type"] = "ALL"
        parts.append(c)

        # 4) ALL_TYPE + ALL_BUILD
        d_all = (w.groupby([gx, gy], as_index=False)
                   .agg(median_price_12m=("price", "median"),
                        sales_12m=("price", "size")))
        d_all["end_month"] = end_month
        d_all["property_type"] = "ALL"
        d_all["new_build"] = "ALL"
        parts.append(d_all)

        print("done:", end_month.date(), "| window rows:", len(w))

    out = pd.concat(parts, ignore_index=True)

    # tidy
    out["end_month"] = pd.to_datetime(out["end_month"]).dt.to_period("M").dt.to_timestamp()
    out["property_type"] = out["property_type"].astype(str)
    out["new_build"] = out["new_build"].astype(str)

    if min_sales > 1:
        out = out[out["sales_12m"] >= min_sales].copy()

    return out

In [ ]:
grid_25km_annual = make_grid_annual_stack_levels(
    df,
    g=25000,
    min_sales=3,
    years_back=10   # latest + last 10 yearly snapshots
)

In [ ]:
grid_10km_annual = make_grid_annual_stack_levels(
    df,
    g=10000,
    min_sales=3,
    years_back=10   # latest + last 10 yearly snapshots
)

In [ ]:
grid_5km_annual = make_grid_annual_stack_levels(
    df,
    g=5000,
    min_sales=3,
    years_back=10   # latest + last 10 yearly snapshots
)

In [ ]:
grid_1km_annual = make_grid_annual_stack_levels(
    df,
    g=1000,
    min_sales=3,
    years_back=1   # latest + last 10 yearly snapshots
)

In [ ]:
grid_1km_annual.to_parquet("/kaggle/working/grid_1km_annual.parquet", index=False)
grid_5km_annual.to_parquet("/kaggle/working/grid_5km_annual.parquet", index=False)
grid_10km_annual.to_parquet("/kaggle/working/grid_10km_annual.parquet", index=False)
grid_25km_annual.to_parquet("/kaggle/working/grid_25km_annual.parquet", index=False)

In [ ]:
import pandas as pd, json, gzip

d = grid_1km_annual.copy()

g = 1000
d = d.rename(columns={
    f"gx_{g}": "gx",
    f"gy_{g}": "gy",
    "median_price_12m": "median",
    "sales_12m": "tx_count",
})

# Normalize end_month to ISO format (YYYY-MM-DD) required for API filtering
d["end_month"] = pd.to_datetime(d["end_month"]).dt.strftime("%Y-%m-%d")

keep = ["gx","gy","end_month","property_type","new_build","median","tx_count"]
d = d[keep].dropna(subset=["gx","gy","median"]).copy()

out_path = "/kaggle/working/grid_1km_full.json.gz"
with gzip.open(out_path, "wt", encoding="utf-8") as f:
    # JSON array (easy to parse/cached in worker)
    json.dump(d.to_dict(orient="records"), f)

print("Wrote:", out_path, "rows:", len(d))

# Extract and store metadata: available date ranges for each dataset
grid_metadata = {}
for grid_size, grid_annual in [
    (1000, grid_1km_annual),
    (5000, grid_5km_annual),
    (10000, grid_10km_annual),
    (25000, grid_25km_annual),
]:
    grid_label = f"{grid_size // 1000}km"
    earliest = pd.to_datetime(grid_annual["end_month"].min()).strftime("%Y-%m-%d")
    latest = pd.to_datetime(grid_annual["end_month"].max()).strftime("%Y-%m-%d")
    grid_metadata[grid_label] = {
        "earliest": earliest,
        "latest": latest,
        "available_months": int(grid_annual["end_month"].nunique())
    }

# Save metadata JSON
metadata_path = "/kaggle/working/grid_metadata.json"
with open(metadata_path, "w") as f:
    json.dump(grid_metadata, f, indent=2)

print(f"\nGrid availability metadata:")
print(json.dumps(grid_metadata, indent=2))
print(f"Saved to {metadata_path}")

In [ ]:
g = 25000

start_month = pd.Timestamp("2025-01-01")
end_month   = pd.Timestamp("2025-12-01")

count = df[
    (df[f"gx_{g}"] == 500000) &
    (df[f"gy_{g}"] == 200000) &
    (df["property_type"] == "D") &
    (df["new_build"] == "Y") &
    (pd.to_datetime(df["month"]).dt.to_period("M").dt.to_timestamp() >= start_month) &
    (pd.to_datetime(df["month"]).dt.to_period("M").dt.to_timestamp() <= end_month)
].shape[0]

count

In [ ]:
(grid_25km_annual["property_type"].eq("ALL") & grid_25km_annual["new_build"].eq("ALL")).any()

In [ ]:
g = 25000

row = grid_25km_annual[
    (grid_25km_annual[f"gx_{g}"] == 500000) &
    (grid_25km_annual[f"gy_{g}"] == 200000) &
    (grid_25km_annual["property_type"] == "D") &
    (grid_25km_annual["new_build"] == "ALL") 
]

row.sort_values(by='sales_12m' , ascending=False)

In [ ]:
import pandas as pd
import numpy as np
from pyproj import Transformer
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "iframe"
TRANSFORMER = Transformer.from_crs("EPSG:27700", "EPSG:4326", always_xy=True)

colorscale = "RdYlGn_r"
lo = 200_000
hi = 1_000_000

def plot_annual_grid_map(
    grid_annual: pd.DataFrame,
    g: int,
    end_month=None,
    property_type: str = "ALL",
    new_build: str = "ALL",
    min_sales: int = 3,
    use_tiles: bool = True,
    zoom: float = 4.6,
    opacity: float = 0.45,
    fill_gaps: bool = True,
    lookback_years: int = 10
):
    def norm_month_series(s: pd.Series) -> pd.Series:
        return pd.to_datetime(s.astype(str)).dt.to_period("M").dt.to_timestamp()

    def norm_month_value(v) -> pd.Timestamp:
        return pd.to_datetime(str(v)).to_period("M").to_timestamp()

    gx_col = f"gx_{g}"
    gy_col = f"gy_{g}"
    cell_col = f"cell_{g}"

    d = grid_annual.copy()
    d["end_month"] = norm_month_series(d["end_month"])

    if end_month is None:
        end_month = d["end_month"].max()
    else:
        end_month = norm_month_value(end_month)

    # Filter to segment first (required)
    d = d[(d["property_type"] == property_type) & (d["new_build"] == new_build)].copy()
    if d.empty:
        raise ValueError("No rows after filtering property_type/new_build. Check your stacked levels exist.")

    # Standardize columns
    d = d.rename(columns={"median_price_12m": "median_price", "sales_12m": "sales"})

    # Restrict to eligible backfill window
    min_allowed = norm_month_value(end_month - pd.DateOffset(years=lookback_years))
    d = d[(d["end_month"] <= end_month) & (d["end_month"] >= min_allowed)].copy()

    if fill_gaps:
        d = (
            d.sort_values("end_month", ascending=False)
             .drop_duplicates(subset=[gx_col, gy_col], keep="first")
             .copy()
        )
        d["end_month_used"] = d["end_month"]
    else:
        d = d[d["end_month"] == end_month].copy()
        d["end_month_used"] = d["end_month"]

    if d.empty:
        raise ValueError("No rows left after applying end_month / backfill window.")

    # Sales threshold
    d = d[d["sales"] >= min_sales].copy()
    if d.empty:
        raise ValueError("No rows meet min_sales after filtering/backfill.")

    d["years_stale"] = (end_month.year - d["end_month_used"].map(lambda t: t.year)).astype(int)

    # Cell id
    d[cell_col] = d[gx_col].astype("int64").astype(str) + "_" + d[gy_col].astype("int64").astype(str)

    # Build GeoJSON squares
    x0 = d[gx_col].astype(float).to_numpy()
    y0 = d[gy_col].astype(float).to_numpy()
    x1 = x0 + g
    y1 = y0 + g

    lon00, lat00 = TRANSFORMER.transform(x0, y0)
    lon10, lat10 = TRANSFORMER.transform(x1, y0)
    lon11, lat11 = TRANSFORMER.transform(x1, y1)
    lon01, lat01 = TRANSFORMER.transform(x0, y1)

    lon_min, lon_max = -10.5, 4.5
    lat_min, lat_max = 49.0, 62.5
    ok = (
        (lon00 >= lon_min) & (lon00 <= lon_max) & (lat00 >= lat_min) & (lat00 <= lat_max) &
        (lon10 >= lon_min) & (lon10 <= lon_max) & (lat10 >= lat_min) & (lat10 <= lat_max) &
        (lon11 >= lon_min) & (lon11 <= lon_max) & (lat11 >= lat_min) & (lat11 <= lat_max) &
        (lon01 >= lon_min) & (lon01 <= lon_max) & (lat01 >= lat_min) & (lat01 <= lat_max)
    )

    d = d.loc[ok].copy()
    if d.empty:
        raise ValueError("Nothing left after UK clipping (check gx/gy values).")

    idx = np.where(ok)[0]
    lon00, lat00 = lon00[idx], lat00[idx]
    lon10, lat10 = lon10[idx], lat10[idx]
    lon11, lat11 = lon11[idx], lat11[idx]
    lon01, lat01 = lon01[idx], lat01[idx]

    ids = d[cell_col].astype(str).to_numpy()

    features = []
    for i in range(len(d)):
        poly = [
            [lon00[i], lat00[i]],
            [lon10[i], lat10[i]],
            [lon11[i], lat11[i]],
            [lon01[i], lat01[i]],
            [lon00[i], lat00[i]],
        ]
        features.append({
            "type": "Feature",
            "id": ids[i],
            "properties": {"cell": ids[i]},
            "geometry": {"type": "Polygon", "coordinates": [poly]},
        })

    geojson = {"type": "FeatureCollection", "features": features}

    lo = float(d["median_price"].quantile(0.05))
    hi = float(d["median_price"].quantile(0.95))

    title = f"UK House Prices ({g//1000}km) — target {end_month.date()} — {property_type}/{new_build}"
    if fill_gaps:
        title += f" (backfilled ≤{lookback_years}y)"

    hover = {
        "median_price": ":,.0f",
        "sales": True,
        "end_month_used": True,
        "years_stale": True,
        "property_type": True,
        "new_build": True,
    }

    if use_tiles:
        fig = px.choropleth_mapbox(
            d,
            geojson=geojson,
            locations=cell_col,
            color="median_price",
            range_color=(lo, hi),
            color_continuous_scale=colorscale,
            hover_data=hover,
            mapbox_style="open-street-map",
            center={"lat": 54.5, "lon": -2.5},
            zoom=zoom,
            opacity=opacity,
            title=title
        )
    else:
        fig = px.choropleth(
            d,
            geojson=geojson,
            locations=cell_col,
            color="median_price",
            range_color=(lo, hi),
            hover_data=hover,
            title=title
        )
        fig.update_geos(fitbounds="locations", visible=False)

    fig.update_traces(marker_line_width=0.3)
    fig.update_layout(margin={"r": 0, "t": 55, "l": 0, "b": 0})
    fig.update_traces(marker_line_width=1.4, marker_line_color="rgba(0,0,0,0.7)")
    fig.show(renderer="iframe")

In [ ]:
g = 25000
grid_annual = grid_25km_annual  # or grid_10km_annual / grid_5km_annual / grid_1km_annual

d_map = plot_annual_grid_map(
    grid_annual,
    g=g,
    end_month="2025-12-01",
    property_type="D",
    new_build="N",
    opacity=0.35,
    fill_gaps=True,
    lookback_years=10
)

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

def build_delta_df(grid_annual: pd.DataFrame, g: int,
                   property_type: str = "ALL", new_build: str = "ALL",
                   min_sales: int = 3,
                   end_month_latest=None,
                   end_month_oldest=None) -> pd.DataFrame:
    gx_col = f"gx_{g}"
    gy_col = f"gy_{g}"
    cell_col = f"cell_{g}"

    d = grid_annual.copy()
    d["end_month"] = pd.to_datetime(d["end_month"].astype(str)).dt.to_period("M").dt.to_timestamp()

    # segment filter (must be explicit to avoid dupes)
    d = d[(d["property_type"] == property_type) & (d["new_build"] == new_build)].copy()
    if d.empty:
        raise ValueError("No rows for that property_type/new_build. (Do you have ALL/ALL stacked?)")

    d = d.rename(columns={"median_price_12m": "median_price", "sales_12m": "sales"})
    d = d[d["sales"] >= min_sales].copy()

    # choose snapshot months
    if end_month_latest is None:
        end_month_latest = d["end_month"].max()
    else:
        end_month_latest = pd.to_datetime(str(end_month_latest)).to_period("M").to_timestamp()

    if end_month_oldest is None:
        end_month_oldest = d["end_month"].min()
    else:
        end_month_oldest = pd.to_datetime(str(end_month_oldest)).to_period("M").to_timestamp()

    latest = d[d["end_month"] == end_month_latest].copy()
    oldest = d[d["end_month"] == end_month_oldest].copy()

    if latest.empty or oldest.empty:
        raise ValueError("Chosen end_month_latest/end_month_oldest not found for that segment.")

    keep = [gx_col, gy_col, "median_price", "sales"]
    latest = latest[keep].rename(columns={"median_price": "price_latest", "sales": "sales_latest"})
    oldest = oldest[keep].rename(columns={"median_price": "price_oldest", "sales": "sales_oldest"})

    out = latest.merge(oldest, on=[gx_col, gy_col], how="inner")

    out["delta_gbp"] = out["price_latest"] - out["price_oldest"]
    out["delta_pct"] = np.where(out["price_oldest"] > 0,
                                (out["price_latest"] / out["price_oldest"] - 1.0) * 100.0,
                                np.nan)

    out[cell_col] = out[gx_col].astype("int64").astype(str) + "_" + out[gy_col].astype("int64").astype(str)
    out["end_month_latest"] = end_month_latest
    out["end_month_oldest"] = end_month_oldest
    out["property_type"] = property_type
    out["new_build"] = new_build

    return out

In [ ]:
from pyproj import Transformer

TRANSFORMER = Transformer.from_crs("EPSG:27700", "EPSG:4326", always_xy=True)

def plot_delta_map(delta_df: pd.DataFrame, g: int,
                   metric: str = "delta_pct",  # "delta_pct" or "delta_gbp"
                   use_tiles: bool = True,
                   zoom: float = 4.6,
                   opacity: float = 0.45):
    gx_col = f"gx_{g}"
    gy_col = f"gy_{g}"
    cell_col = f"cell_{g}"

    d = delta_df.copy()

    # GeoJSON squares
    x0 = d[gx_col].astype(float).to_numpy()
    y0 = d[gy_col].astype(float).to_numpy()
    x1 = x0 + g
    y1 = y0 + g

    lon00, lat00 = TRANSFORMER.transform(x0, y0)
    lon10, lat10 = TRANSFORMER.transform(x1, y0)
    lon11, lat11 = TRANSFORMER.transform(x1, y1)
    lon01, lat01 = TRANSFORMER.transform(x0, y1)

    # UK-ish clip
    ok = (
        (lon00 >= -10.5) & (lon00 <= 4.5) & (lat00 >= 49.0) & (lat00 <= 62.5) &
        (lon10 >= -10.5) & (lon10 <= 4.5) & (lat10 >= 49.0) & (lat10 <= 62.5) &
        (lon11 >= -10.5) & (lon11 <= 4.5) & (lat11 >= 49.0) & (lat11 <= 62.5) &
        (lon01 >= -10.5) & (lon01 <= 4.5) & (lat01 >= 49.0) & (lat01 <= 62.5)
    )
    d = d.loc[ok].copy()
    idx = np.where(ok)[0]
    lon00, lat00 = lon00[idx], lat00[idx]
    lon10, lat10 = lon10[idx], lat10[idx]
    lon11, lat11 = lon11[idx], lat11[idx]
    lon01, lat01 = lon01[idx], lat01[idx]

    ids = d[cell_col].astype(str).to_numpy()
    features = []
    for i in range(len(d)):
        poly = [[lon00[i], lat00[i]],[lon10[i], lat10[i]],[lon11[i], lat11[i]],[lon01[i], lat01[i]],[lon00[i], lat00[i]]]
        features.append({"type":"Feature","id":ids[i],"properties":{"cell":ids[i]},
                         "geometry":{"type":"Polygon","coordinates":[poly]}})
    geojson = {"type":"FeatureCollection","features":features}

    end_latest = pd.to_datetime(d["end_month_latest"].iloc[0]).date()
    end_oldest = pd.to_datetime(d["end_month_oldest"].iloc[0]).date()
    seg = f"{d['property_type'].iloc[0]}/{d['new_build'].iloc[0]}"

    # Diverging colour scale for deltas
    colorscale = "RdBu"  # red=negative, blue=positive by default
    if metric == "delta_pct":
        # centre at 0 using symmetric range based on 95th percentile
        m = float(np.nanpercentile(np.abs(d["delta_pct"]), 95))
        rng = (-m, m)
        title = f"Δ% (12m ending {end_oldest} → {end_latest}) — {seg}"
        hover = {"delta_pct":":.1f","delta_gbp":":,.0f","price_oldest":":,.0f","price_latest":":,.0f",
                 "sales_oldest":True,"sales_latest":True}
        color_col = "delta_pct"
    else:
        m = float(np.nanpercentile(np.abs(d["delta_gbp"]), 95))
        rng = (-m, m)
        title = f"Δ£ (12m ending {end_oldest} → {end_latest}) — {seg}"
        hover = {"delta_gbp":":,.0f","delta_pct":":.1f","price_oldest":":,.0f","price_latest":":,.0f",
                 "sales_oldest":True,"sales_latest":True}
        color_col = "delta_gbp"

    if use_tiles:
        fig = px.choropleth_mapbox(
            d, geojson=geojson, locations=cell_col, color=color_col,
            range_color=rng, color_continuous_scale=colorscale,
            hover_data=hover, mapbox_style="open-street-map",
            center={"lat":54.5,"lon":-2.5}, zoom=zoom, opacity=opacity, title=title
        )
    else:
        fig = px.choropleth(
            d, geojson=geojson, locations=cell_col, color=color_col,
            range_color=rng, color_continuous_scale=colorscale,
            hover_data=hover, title=title
        )
        fig.update_geos(fitbounds="locations", visible=False)

    # darker borders like you wanted
    fig.update_traces(marker_line_width=1.1, marker_line_color="rgba(0,0,0,0.9)")
    fig.update_layout(margin={"r":0,"t":55,"l":0,"b":0})
    fig.show(renderer="iframe")

In [ ]:
def plot_top_movers(delta_df: pd.DataFrame, n: int = 20):
    d = delta_df.dropna(subset=["delta_pct"]).copy()
    d = d.sort_values("delta_pct", ascending=False).head(n).copy()

    fig = px.bar(
        d,
        x="delta_pct",
        y=d.index.astype(str),
        hover_data={"delta_pct":":.1f","delta_gbp":":,.0f","price_oldest":":,.0f","price_latest":":,.0f"},
        title=f"Top {n} grid cells by Δ% (joined cells only)"
    )
    fig.update_layout(yaxis_title="row", xaxis_title="Δ%")
    fig.show(renderer="iframe")

In [ ]:
def build_overall_deltas(grid_annual: pd.DataFrame, min_sales: int = 3) -> pd.DataFrame:
    """
    Compute deltas between earliest and latest available month across entire dataset.
    
    KAGGLE OPTIMIZATION:
    - Avoids storing all intermediate period deltas; only computes earliest→latest
    - Single merge operation (lower memory footprint than multi-period approach)
    - Filtered by min_sales to reduce output size
    - Returns only non-null deltas to save storage
    
    Args:
        grid_annual: Annual stacked grid data with columns [gx_*, gy_*, end_month, property_type, new_build, median, sales_12m]
        min_sales: Minimum transaction count to include
    
    Returns:
        DataFrame with columns [gx, gy, cell, property_type, new_build, 
        price_earliest, sales_earliest, end_month_earliest,
        price_latest, sales_latest, end_month_latest,
        delta_gbp, delta_pct, years_delta]
    """
    d = grid_annual.copy()
    print(f"DEBUG: build_overall_deltas input: {len(d)} rows, min_sales filter={min_sales}")
    d = d[d["sales_12m"] >= min_sales].copy()
    print(f"DEBUG: after min_sales filter: {len(d)} rows")
    
    if d.empty:
        return pd.DataFrame()
    
    # Find earliest and latest months available
    earliest_month = d["end_month"].min()
    latest_month = d["end_month"].max()
    
    print(f"DEBUG: earliest_month={earliest_month}, latest_month={latest_month}")
    
    if earliest_month == latest_month:
        print(f"Only one month available ({earliest_month}); skipping overall deltas")
        return pd.DataFrame()
    
    # Subset to earliest and latest only (memory efficient)
    earliest_data = d[d["end_month"] == earliest_month].copy()
    latest_data = d[d["end_month"] == latest_month].copy()
    
    # Determine grid size from column names (needed for debug output)
    grid_sizes = [g for col in earliest_data.columns if col.startswith("gx_") for g in [int(col.split("_")[1])]]
    if not grid_sizes:
        raise ValueError("No grid columns found (expected gx_*, gy_*)")
    g = grid_sizes[0]
    
    gx_col = f"gx_{g}"
    gy_col = f"gy_{g}"
    cell_col = f"cell_{g}"
    
    print(f"DEBUG earliest_data rows: {len(earliest_data)}, unique cells: {earliest_data[[gx_col, gy_col]].drop_duplicates().shape[0]}")
    print(f"DEBUG latest_data rows: {len(latest_data)}, unique cells: {latest_data[[gx_col, gy_col]].drop_duplicates().shape[0]}")
    
    # Group by cell, property_type, new_build at each time point
    earliest_agg = earliest_data.groupby([gx_col, gy_col, "property_type", "new_build"]).agg({
        "median_price_12m": "first",
        "sales_12m": "first"
    }).reset_index().rename(columns={"median_price_12m": "price_earliest", "sales_12m": "sales_earliest"})
    
    latest_agg = latest_data.groupby([gx_col, gy_col, "property_type", "new_build"]).agg({
        "median_price_12m": "first",
        "sales_12m": "first"
    }).reset_index().rename(columns={"median_price_12m": "price_latest", "sales_12m": "sales_latest"})
    
    print(f"DEBUG: earliest_agg={len(earliest_agg)} unique cell+segment combos")
    print(f"DEBUG: latest_agg={len(latest_agg)} unique cell+segment combos")
    
    # Inner join: only cells with data in both periods
    out = earliest_agg.merge(
        latest_agg,
        on=[gx_col, gy_col, "property_type", "new_build"],
        how="inner"
    )
    
    if out.empty:
        print(f"No cells with data in both {earliest_month} and {latest_month}")
        return pd.DataFrame()
    
    # DEBUG: Show coordinate distribution
    unique_cells = out[[gx_col, gy_col]].drop_duplicates()
    print(f"DEBUG: {len(unique_cells)} unique grid cells")
    print(f"DEBUG: {gx_col} range: {out[gx_col].min()} to {out[gx_col].max()}")
    print(f"DEBUG: {gy_col} range: {out[gy_col].min()} to {out[gy_col].max()}")
    if len(unique_cells) <= 10:
        print(f"DEBUG: Cells - {unique_cells.values.tolist()}")
    
    # Compute deltas
    out["delta_gbp"] = out["price_latest"] - out["price_earliest"]
    out["delta_pct"] = np.where(
        out["price_earliest"] > 0,
        (out["price_latest"] / out["price_earliest"] - 1.0) * 100.0,
        np.nan
    )
    
    # Add metadata
    out[cell_col] = out[gx_col].astype("int64").astype(str) + "_" + out[gy_col].astype("int64").astype(str)
    out["end_month_earliest"] = earliest_month
    out["end_month_latest"] = latest_month
    out["years_delta"] = (pd.to_datetime(latest_month).year - pd.to_datetime(earliest_month).year)
    # Also expose generic coordinate columns and a generic cell id for JSON consumers
    out["gx"] = out[gx_col].astype(float)
    out["gy"] = out[gy_col].astype(float)
    out["cell"] = out[cell_col].astype(str)
    
    # Select output columns (clean up grid-specific columns)
    # NOTE: sales_latest retained to contextualize large delta_pct values (sparse vs. robust moves)
    keep_cols = [
        gx_col, gy_col, "gx", "gy", cell_col, "cell", "property_type", "new_build",
        "price_earliest", "sales_earliest", "end_month_earliest",
        "price_latest", "sales_latest", "end_month_latest",
        "delta_gbp", "delta_pct", "years_delta"
    ]
    out = out[[c for c in keep_cols if c in out.columns]]
    
    # Filter to non-null deltas only (saves storage)
    out = out.dropna(subset=["delta_pct"])
    
    # Convert Timestamps to ISO format strings for JSON serialization
    out["end_month_earliest"] = pd.to_datetime(out["end_month_earliest"]).dt.strftime("%Y-%m-%d")
    out["end_month_latest"] = pd.to_datetime(out["end_month_latest"]).dt.strftime("%Y-%m-%d")
    
    return out.sort_values("delta_pct", ascending=False)

In [ ]:
g = 5000
grid_annual = grid_5km_annual

delta_25 = build_delta_df(
    grid_annual, g,
    property_type="D",
    new_build="ALL",
    min_sales=30
)

plot_delta_map(delta_25, g, metric="delta_pct", opacity=0.6)

## Computing Overall Deltas (Earliest → Latest)

For Kaggle efficiency, use `build_overall_deltas()` to compute deltas across your entire dataset history.
This single comparison is much cheaper than computing all period-to-period deltas.

In [ ]:
# Example: Compute overall deltas for all property types across 5km, 10km, 25km grids
# (Skip 1km to avoid excessive data volume)

delta_metadata = {}

for grid_size, grid_annual in [
    (5000, grid_5km_annual),
    (10000, grid_10km_annual),
    (25000, grid_25km_annual),
]:
    grid_label = f"{grid_size // 1000}km"
    overall_deltas = build_overall_deltas(grid_annual, min_sales=5)
    
    if not overall_deltas.empty:
        cell_col = f"cell_{grid_size}"
        print(f"\n{'='*60}")
        print(f"Overall Deltas: {grid_label}")
        print(f"{'='*60}")
        print(f"Generated {len(overall_deltas)} cells with price movement")
        print(f"Time range: {overall_deltas['end_month_earliest'].iloc[0]} → {overall_deltas['end_month_latest'].iloc[0]}")
        print(f"Years: {overall_deltas['years_delta'].iloc[0]}")
        print(f"\nTop 10 gainers (%):")
        print(overall_deltas.nlargest(10, "delta_pct")[[cell_col, "property_type", "new_build", "sales_latest", "delta_pct", "delta_gbp"]])
        
        # Ensure grid coordinate columns are native Python types and add lon/lat center
        gx_col = f"gx_{grid_size}"
        gy_col = f"gy_{grid_size}"
        try:
            overall_deltas[gx_col] = overall_deltas[gx_col].astype("int64")
            overall_deltas[gy_col] = overall_deltas[gy_col].astype("int64")
        except Exception:
            # best-effort cast
            overall_deltas[gx_col] = overall_deltas[gx_col].astype(float).round(0).astype("Int64")
            overall_deltas[gy_col] = overall_deltas[gy_col].astype(float).round(0).astype("Int64")

        overall_deltas["gx"] = overall_deltas["gx"].astype(float).round(0).astype("Int64")
        overall_deltas["gy"] = overall_deltas["gy"].astype(float).round(0).astype("Int64")

        # Add centre lon/lat for convenience (EPSG:27700 -> EPSG:4326)
        try:
            x_cent = overall_deltas[gx_col].astype(float).to_numpy() + (grid_size / 2.0)
            y_cent = overall_deltas[gy_col].astype(float).to_numpy() + (grid_size / 2.0)
            lon_c, lat_c = TRANSFORMER.transform(x_cent, y_cent)
            overall_deltas["lon"] = lon_c
            overall_deltas["lat"] = lat_c
        except Exception:
            overall_deltas["lon"] = None
            overall_deltas["lat"] = None

        # Convert NaNs to None for JSON and ensure timestamps are strings
        overall_deltas["end_month_earliest"] = overall_deltas["end_month_earliest"].astype(str)
        overall_deltas["end_month_latest"] = overall_deltas["end_month_latest"].astype(str)

        # Capture metadata for this grid
        earliest = overall_deltas["end_month_earliest"].iloc[0]
        latest = overall_deltas["end_month_latest"].iloc[0]
        delta_metadata[grid_label] = {
            "earliest": earliest,
            "latest": latest,
            "rows": len(overall_deltas)
        }

        # Save to JSON (memory-efficient for Kaggle)
        output_path = f"/kaggle/working/deltas_overall_{grid_label}.json.gz"
        with gzip.open(output_path, "wt", encoding="utf-8") as f:
            json.dump(overall_deltas.where(pd.notnull(overall_deltas), None).to_dict(orient="records"), f)
        print(f"\nSaved to {output_path}")
    else:
        print(f"\nNo deltas generated for {grid_label}")

# Save delta metadata file
if delta_metadata:
    delta_metadata_path = "/kaggle/working/deltas_metadata.json"
    with open(delta_metadata_path, "w") as f:
        json.dump(delta_metadata, f, indent=2)
    print(f"\n{'='*60}")
    print("Delta metadata:")
    print(json.dumps(delta_metadata, indent=2))
    print(f"Saved to {delta_metadata_path}")
#plot_top_movers(delta_25, n=20)